In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re

In [4]:
dataset = pd.read_csv('/content/tamil_offensive_train.csv')

In [5]:
dataset.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [6]:
dataset.shape

(5880, 3)

In [7]:
dataset.drop(labels='id', axis=1, inplace=True)

In [8]:
dataset.head()

,text,category
0,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,only for விஜய் சேதுபதி and STR,NOT


In [9]:
dataset.category.unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [10]:
dataset = dataset.loc[dataset["category"] != 'not-Tamil']

In [11]:
dataset.category.unique()

array(['NOT', 'OFF'], dtype=object)

In [12]:
dataset['category'] = dataset.category.replace(to_replace=['NOT', 'OFF'], value=[0, 1])
# 0 - Not offensive
# 1 - Offensive

In [13]:
dataset.head()

,text,category
0,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,0
1,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,1
2,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,0
3,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,1
4,only for விஜய் சேதுபதி and STR,0


In [14]:
dataset.text[0]

'திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்'

In [15]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [16]:
import requests

link = "https://raw.githubusercontent.com/AshokR/TamilNLP/master/tamilnlp/Resources/TamilStopWords.txt"
req = requests.get(link)
stopwords = req.text

In [17]:
def clean_text(text):

  corpus = ''.join(str(j) for j in text)
  corpus = corpus.replace('.','')
  corpus = corpus.replace('?','')
  corpus = corpus.replace(',','')
  corpus = corpus.replace('!','')
  corpus = corpus.replace('/','')
  corpus = corpus.replace('@','')
  corpus = corpus.replace('\\','')
  corpus = corpus.replace('(','')
  corpus = corpus.replace(')','')
  corpus = corpus.replace('-','')
  corpus = corpus.replace('_','')
  corpus = corpus.replace('@','')
  corpus = corpus.replace('#','')
  corpus = corpus.replace('$','')
  corpus = corpus.replace(';','')
  corpus = corpus.replace(':','')
  corpus = corpus.replace('\'','')
  corpus = corpus.replace('\"','')
  corpus = corpus.replace('*','')
  corpus = ''.join(c for c in corpus if not c.isdigit())
  corpus = deEmojify(corpus)
  corpus = corpus.split()
  corpus = [word for word in corpus if word not in str(stopwords.split()) and not word.isalpha()]
  corpus = ' '.join(corpus)
  return corpus

In [18]:
for row in dataset:
  dataset['text'] = dataset['text'].apply(clean_text)

In [19]:
dataset.head()

,text,category
0,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,0
1,ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை தியேட்டர்...,1
2,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,0
3,மொத்த சாதியும் சாதிக்கு எதிரா நிக்குது என்னா ச...,1
4,விஜய் சேதுபதி,0


In [23]:
dataset.shape

(5877, 2)

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1), min_df=50)
dtm = vect.fit_transform(dataset.text)

In [105]:
dtm.shape

(5877, 192)

In [ ]:
vect.get_feature_names()

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dtm, dataset['category'], test_size=.25)

In [108]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((4407, 192), (4407,), (1470, 192), (1470,))

In [109]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train.toarray(), y_train)
nb_pred_train = gnb.predict(X_train.toarray())
nb_pred_test = gnb.predict(X_test.toarray())

In [110]:
nb_pred_test

array([1, 0, 0, ..., 1, 0, 1])

In [111]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,nb_pred_test)

array([[781, 414],
       [ 82, 193]])

In [112]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,nb_pred_test)

0.6625850340136055

In [113]:
from sklearn.metrics import classification_report
print(classification_report(y_test,nb_pred_test))

              precision    recall  f1-score   support

           0       0.90      0.65      0.76      1195
           1       0.32      0.70      0.44       275

    accuracy                           0.66      1470
   macro avg       0.61      0.68      0.60      1470
weighted avg       0.80      0.66      0.70      1470



In [114]:
from sklearn.metrics import f1_score
f1_score(y_test,nb_pred_test,average='weighted')

0.6988732648874256

In [180]:
test_df = pd.read_csv('/content/tamil_test.csv')
test_df.head()

,id\ttext,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,tam_test_1\tதலைவா STR இதுக்குதான் கதுருந்தோம் ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tam_test_2\tநாயுடு மக்கள் சார்பாக திரைப்படம் வ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tam_test_3\tதில்லானா முயற்சி தஞ்சை கோனார்,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tam_test_4\tதிரௌபதியின் துகிலுறித்த போது காத்த...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tam_test_5\tநான் தியேட்டர்லே படம் பார்த்து 35 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
test_df.shape

(654, 16)

In [182]:
test_df.columns

Index(['id\ttext', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15'],
      dtype='object')

In [183]:
test_df.drop(labels=(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15']),axis=1,inplace=True)

In [184]:
test_df.head()

,id\ttext
0,tam_test_1\tதலைவா STR இதுக்குதான் கதுருந்தோம் ...
1,tam_test_2\tநாயுடு மக்கள் சார்பாக திரைப்படம் வ...
2,tam_test_3\tதில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4\tதிரௌபதியின் துகிலுறித்த போது காத்த...
4,tam_test_5\tநான் தியேட்டர்லே படம் பார்த்து 35 ...


In [185]:
for row in dataset:
  test_df['id\ttext']= test_df['id\ttext'].apply(clean_text)

In [187]:
test_df.rename(columns={'id\ttext': 'text'},inplace=True)

In [195]:
test_vect = TfidfVectorizer(ngram_range=(1,1), max_features=192)
test_dtm = test_vect.fit_transform(test_df.text)

In [196]:
test_dtm.shape,dtm.shape

((654, 192), (5877, 192))

In [197]:
pred_test = gnb.predict(test_dtm.toarray())

In [201]:
test_df['Prediction'] = pred_test

In [202]:
test_df.head()

,text,Prediction
0,தலைவா இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா டீம்,0
1,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...,1
2,தில்லானா முயற்சி தஞ்சை கோனார்,0
3,திரௌபதியின் துகிலுறித்த காத்த கண்ணனின் யாதவ கு...,1
4,தியேட்டர்லே படம் பார்த்து வருஷமாச்சு எப்படா தி...,0


In [204]:
test_df.to_csv('/content/Final_test-Results.csv')